In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import warnings

import sys

sys.path.insert(0, "../code_workspace")

from data_prep import *
from post_analysis import *

warnings.filterwarnings("ignore")

/Users/suatakbulut/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<IPython.core.display.Javascript object>

In [3]:
use_saved_one = True
unwanted_column_endings = []
test_size = 0.4
models_path = "../trained_models/results_df_0.77_AUC_0.61_Rec.pkl"

<IPython.core.display.Javascript object>

In [4]:
X_train, X_test, y_train, y_test = prepare_train_test_split(
    use_saved_one=use_saved_one,
    unwanted_column_endings=unwanted_column_endings,
    test_size=test_size,
)


DATA PREP

Loading processed data from ../data/processed_dataset.pkl..
Excluding the following columns..
['patient_id', 'Outcome 30 days Hospitalization', 'Height(in)', 'Weight(lbs)']
Shapes: X_train: (108861, 136) - X_test: (72574, 136)


<IPython.core.display.Javascript object>

In [5]:
if not os.path.exists(models_path):
    raise ValueError(f"Trained model path {model_path} does not exist.")

results_df = pd.read_pickle(models_path)
results_df.set_index("classifier_class", drop=True)[
    ["AUC", "Recall", "Precision", "F1_Score", "Accuracy"]
]

,AUC,Recall,Precision,F1_Score,Accuracy
classifier_class,,,,,
RandomForestClassifier,0.769360,0.606889,0.674594,0.638952,0.706686
DecisionTreeClassifier,0.734679,0.566614,0.644530,0.603066,0.681015
GaussianNB,0.652868,0.560331,0.552130,0.556201,0.617590
QuadraticDiscriminantAnalysis,0.693367,0.521378,0.597497,0.556848,0.645107
GradientBoostingClassifier,0.750243,0.479879,0.695461,0.567899,0.687698
LogisticRegression,0.712588,0.449399,0.651015,0.531737,0.661504


<IPython.core.display.Javascript object>

In [6]:
best_model = (
    results_df[results_df.classifier_class == "RandomForestClassifier"]
    .trained_model.values[0]
    .best_estimator_
)

best_predictor = best_model.named_steps["predictor"]

<IPython.core.display.Javascript object>

In [7]:
_, _, feature_importance_labels = feature_importance_top10(best_model, X_test, y_test)
# _, permutation_importance_labels = permutation_importance_top10(best_model, X_test, y_test)
# imp_features = list(set(feature_importance_labels).union(set(permutation_importance_labels)))
imp_features = feature_importance_labels

Elapsed time to compute feature importances: 0.00 mins


<IPython.core.display.Javascript object>

In [8]:
preprocessor = get_preprocessor()

X_train = preprocessor.fit_transform(X_train[imp_features])
X_test = preprocessor.transform(X_test[imp_features])

logit = sm.Logit(y_train, sm.add_constant(X_train)).fit()

Optimization terminated successfully.
         Current function value: 0.634368
         Iterations 5


<IPython.core.display.Javascript object>

In [9]:
print_logit_results(logit, X_train, X_test, y_train, y_test, imp_features)

                                  Logit Regression Results                                 
Dep. Variable:     Outcome 30 days Hospitalization   No. Observations:               108861
Model:                                       Logit   Df Residuals:                   108850
Method:                                        MLE   Df Model:                           10
Date:                             Mon, 14 Aug 2023   Pseudo R-squ.:                 0.06928
Time:                                     22:17:39   Log-Likelihood:                -69058.
converged:                                    True   LL-Null:                       -74199.
Covariance Type:                         nonrobust   LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0.3415      0.007    -52.452      0.000      -0.354    

<IPython.core.display.Javascript object>

### Re-fit the best model with the selected features only

In [10]:
best_predictor.fit(X_train, y_train)

rf_auc_score, rf_f_score, rf_accuracy, rf_precision, rf_recall = calculate_metrics(
    best_predictor, X_test, y_test
)
(
    rf_auc_score_tr,
    rf_f_score_tr,
    rf_accuracy_tr,
    rf_precision_tr,
    rf_recall_tr,
) = calculate_metrics(best_predictor, X_train, y_train)

<IPython.core.display.Javascript object>

In [11]:
## Get scores for Logit on Test and Train sets
(
    log_auc_score,
    log_f_score,
    log_accuracy,
    log_precision,
    log_recall,
) = calculate_stats_logit_metrics(logit, X_test, y_test)
(
    log_auc_score_tr,
    log_f_score_tr,
    log_accuracy_tr,
    log_precision_tr,
    log_recall_tr,
) = calculate_stats_logit_metrics(logit, X_train, y_train)

<IPython.core.display.Javascript object>

In [12]:
# Combine scores
forest_dict = {
    "AUC": [f"{rf_auc_score:.4f}", f"{rf_auc_score_tr:.4f}"],
    "Recall": [f"{rf_recall:.4f}", f"{rf_recall_tr:.4f}"],
    "F1": [f"{rf_f_score:.4f}", f"{rf_f_score_tr:.4f}"],
    "Accuracy": [f"{rf_accuracy:.4f}", f"{rf_accuracy_tr:.4f}"],
    "Precision": [f"{rf_precision:.4f}", f"{rf_precision_tr:.4f}"],
}
forest_res = pd.DataFrame(
    forest_dict, index=["Forest on Test", "Forest on Train"]
).transpose()

logit_dict = {
    "AUC": [f"{log_auc_score:.4f}", f"{log_auc_score_tr:.4f}"],
    "Recall": [f"{log_recall:.4f}", f"{log_recall_tr:.4f}"],
    "F1": [f"{log_f_score:.4f}", f"{log_f_score_tr:.4f}"],
    "Accuracy": [f"{log_accuracy:.4f}", f"{log_accuracy_tr:.4f}"],
    "Precision": [f"{log_precision:.4f}", f"{log_precision_tr:.4f}"],
}
logit_res = pd.DataFrame(
    logit_dict, index=["Logit on Test", "Logit on Train"]
).transpose()


comparison = pd.concat([logit_res, forest_res], axis=1)
comparison

,Logit on Test,Logit on Train,Forest on Test,Forest on Train
AUC,0.6780,0.6807,0.7338,0.9300
Recall,0.3786,0.3837,0.5649,0.7769
F1,0.4680,0.4710,0.6006,0.8146
Accuracy,0.6319,0.6345,0.6786,0.8500
Precision,0.6127,0.6099,0.6410,0.8562


<IPython.core.display.Javascript object>